Задание 5. Web-scraping на языке Python

Используя доступ к данным по API, соберите с [ресурса](https://trudvsem.ru/) (https://trudvsem.ru/) данные по интересующим Вас вакансиям или вакансиям Вашего региона.
Дополнительную информацию  по данному API можно найти [здесь](https://trudvsem.ru/opendata/api). Количество записей и набор признаков определяете самостоятельно. Собранные данные сохраните в DataFrame.  


подключение библиотек

In [71]:
import requests
import pandas as pd
import json
import csv
import os

 отправка GET-запроса

In [24]:
api_url = f"http://opendata.trudvsem.ru/api/v1/vacancies/region/64"

In [25]:
answer = requests.get(api_url)

конвертация ответа сервера в удобный формат-- json

In [26]:
vacancies = answer.json()
json.loads(answer.text)

{'status': '200',
 'request': {'api': 'v1'},
 'meta': {'total': 8729, 'limit': 100},
 'results': {'vacancies': [{'vacancy': {'id': '30fdc348-622e-11f0-a151-d549be31d974',
     'source': 'Работодатель',
     'region': {'region_code': '6400000000000', 'name': 'Саратовская область'},
     'company': {'companycode': '1026402484591',
      'email': 'gro@oaosaratovgaz.ru',
      'hr-agency': False,
      'inn': '6451114530',
      'kpp': '645101001',
      'name': 'АО "САРАТОВГАЗ"',
      'ogrn': '1026402484591',
      'url': 'https://trudvsem.ru/company/1026402484591'},
     'creation-date': '2025-07-16',
     'salary': 'от 28050',
     'salary_min': 28050,
     'salary_max': 36400,
     'job-name': 'Штукатур 5 разряда Отдела капитального строительства',
     'vac_url': 'https://trudvsem.ru/vacancy/card/1026402484591/30fdc348-622e-11f0-a151-d549be31d974',
     'employment': 'Полная занятость',
     'schedule': 'Полный рабочий день',
     'duty': 'Должностные обязанности согласно должностной

анализ структуры полученного ответа

In [27]:
vacancies.keys()

dict_keys(['status', 'request', 'meta', 'results'])

Анализ содержимого по ключам

In [29]:
vacancies.get('status')
vacancies.get('request')
vacancies.get('meta')
vacancies.get('results')

{'vacancies': [{'vacancy': {'id': '30fdc348-622e-11f0-a151-d549be31d974',
    'source': 'Работодатель',
    'region': {'region_code': '6400000000000', 'name': 'Саратовская область'},
    'company': {'companycode': '1026402484591',
     'email': 'gro@oaosaratovgaz.ru',
     'hr-agency': False,
     'inn': '6451114530',
     'kpp': '645101001',
     'name': 'АО "САРАТОВГАЗ"',
     'ogrn': '1026402484591',
     'url': 'https://trudvsem.ru/company/1026402484591'},
    'creation-date': '2025-07-16',
    'salary': 'от 28050',
    'salary_min': 28050,
    'salary_max': 36400,
    'job-name': 'Штукатур 5 разряда Отдела капитального строительства',
    'vac_url': 'https://trudvsem.ru/vacancy/card/1026402484591/30fdc348-622e-11f0-a151-d549be31d974',
    'employment': 'Полная занятость',
    'schedule': 'Полный рабочий день',
    'duty': 'Должностные обязанности согласно должностной инструкции, соблюдение внутреннего трудового распорядка',
    'category': {'specialisation': 'Химическая, нефтехими

посчитать кол-во запросов чтобы получить все данные

In [30]:
count_request = vacancies.get('meta').get('total') // vacancies.get('meta').get('limit') + 1
count_request

88

отдельная вакансия

In [31]:
vacancies.get('results').get('vacancies')[0].get('')

Разбор отделной вакансии

In [69]:
vacancies.get('results').get('vacancies')[0].keys()
vacancies.get('results').get('vacancies')[0].get('vacancy').keys()
vacancies.get('results').get('vacancies')[0].get('vacancy').get('company')
vacancies.get('results').get('vacancies')[36].get('vacancy').get('contact_list')

[{'contact_type': 'Телефон', 'contact_value': '8452691018'},
 {'contact_type': 'Эл. почта', 'contact_value': 'O.Khohlova@oaosaratovgaz.ru'}]

### соберем набор данных:
- id

- region_name

- company_name

- company_inn

- salary

- salary_min

- salary_max

- job_name

- phone

- mobile_phone

- email

- duty


In [70]:
name_file = 'vacancy.csv'
all_vac = []
dct_vac = dict.fromkeys(['vac_id', 'vac_region_name', 'job_name', 'salary', 'salary_min',
                        'salary_max', 'vac_company_name', 'vac_company_inn',
                        'phone', 'mobile_phone', 'email', 'duty'], None)

for vac in vacancies.get('results').get('vacancies'):
    dct_vac['vac_id'] = vac.get('vacancy').get('id')
    dct_vac['vac_region_name'] = vac.get('vacancy').get('region').get('name', 'данных нет')
    dct_vac['vac_company_name'] = vac.get('vacancy').get('company').get('name', 'данных нет')
    dct_vac['vac_company_inn'] = vac.get('vacancy').get('company').get('inn', 'данных нет')
    dct_vac['job_name'] = vac.get('vacancy').get('job-name', 'данных нет')
    dct_vac['salary'] = vac.get('vacancy').get('salary', 'данных нет')
    dct_vac['salary_min'] = vac.get('vacancy').get('salary_min', 'данных нет')
    dct_vac['salary_max'] = vac.get('vacancy').get('salary_max', 'данных нет')
    dct_vac['duty'] = vac.get('vacancy').get('duty', 'данных нет')
    dct_vac['phone'] = None
    dct_vac['mobile_phone'] = None
    dct_vac['email'] = None
    contact_list = vac.get('vacancy').get('contact_list', [])
    for contact in contact_list:
        contact_type = contact.get('contact_type', 'данных нет').lower()
        if not dct_vac['phone'] and 'телефон' in contact_type and 'мобильный' not in contact_type:
            dct_vac['phone'] = contact.get('contact_value')
        elif not dct_vac['mobile_phone'] and 'мобильный телефон' in contact_type:
            dct_vac['mobile_phone'] = contact.get('contact_value')
        elif not dct_vac['email'] and ('почта' in contact_type or 'email' in contact_type):
            dct_vac['email'] = contact.get('contact_value')
    
    
    if os.path.isfile(name_file):
        with open(name_file, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = list(dct_vac.keys())
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(dct_vac.copy())
    else:
        with open(name_file, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = list(dct_vac.keys())
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow(dct_vac.copy())
    all_vac.append(dct_vac.copy())

создаем DataFrame


In [72]:
df_vacancy = pd.DataFrame(all_vac)
df_vacancy.head(25)

,vac_id,vac_region_name,job_name,salary,salary_min,salary_max,vac_company_name,vac_company_inn,phone,mobile_phone,email,duty
0,30fdc348-622e-11f0-a151-d549be31d974,Саратовская область,Штукатур 5 разряда Отдела капитального строите...,от 28050,28050,36400,"АО ""САРАТОВГАЗ""",6451114530,None,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
1,dc582465-afa6-11ee-bc12-bb4c34bddc95,Саратовская область,Инженер 1 категории Отдела подготовки и провед...,от 34330,34330,49700,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
2,8aa258d0-6b04-11ed-9f4b-17f530626801,Саратовская область,Электрогазосварщик 4 разряда транспортной службы,от 26060,26060,33870,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
3,108d3448-623a-11f0-ac45-cb26dff57dd7,Саратовская область,Слесарь по ремонту автомобилей 3 разряда Тран...,от 23000,23000,25500,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
4,c926e700-6224-11f0-ac45-cb26dff57dd7,Саратовская область,Курьер Службы эксплуатации пунктов редуцирован...,от 23000,23000,23000,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
5,f243f308-7b0f-11ef-8067-cb26dff57dd7,Саратовская область,Подсобный рабочий 3 разряда Отдел капитального...,от 23000,23000,29350,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
6,151dc990-4a78-11f0-9276-d549be31d974,Саратовская область,Инженер-энергетик,от 42000,42000,42000,"ОАО ""САРАТОВСКИЙ ЗАВОД ""СЕРП И МОЛОТ""",6455003006,+7(845) 247-13-97,None,personal@sarsim.ru,"оптовый рынок ээ (заявки, формирование отчетов..."
7,0f070310-6232-11f0-a151-d549be31d974,Саратовская область,Мойщик посуды 2 разряда Столовой,от 23000,23000,23000,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
8,6bb2c2c5-9620-11ec-92fd-550ed7335bbe,Саратовская область,Газосварщик 5 разряда строительно-монтажного у...,от 33730,33730,43840,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...
9,0a0d0e48-66bb-11ef-94ce-d549be31d974,Саратовская область,Слесарь аварийно-восстановительных работ 6 раз...,от 30430,30430,39550,"АО ""САРАТОВГАЗ""",6451114530,+7(845) 269-10-18,None,O.Khohlova@oaosaratovgaz.ru,Должностные обязанности согласно должностной и...


# Написание кода для скрейпинга в виде функций:
1. функция -- отправка запроса с параметрами

In [77]:
def send_get_request(api_url: str, params_api: dict=None) -> dict:
    """
    Функция отправляет get-запрос по указанному адресу с заданными параметрами

    Args:
        api_url: str -- адрес API
        params_api: dict -- параметры запроса

    Return:
        Функция возвращает ответ сервера в формате json.
    """
    response = requests.get(api_url, params=params_api)
    if response.status_code == 200:
        print('Успешно, запрос отправлен на: ', response.url)
    else:
        print('Ошибка: ', response.status_code)
    return response.json()

2. Функция -- список вакансий

In [76]:
def get_list_vac(all_vac:dict) -> list:
    """
    Извлекает список вакансий из словаря all_vac,
    который содержит результат запроса к API.
    Args:
        all_vac (dict): словарь с данными
    """
    return all_vac.get('results').get('vacancies')

3. Функция -- разбор ответа

In [75]:
def get_vacansy(vac_info: dict):
    """
    Функция извлекает и структурирует информацию о вакансии из словаря vac_info.
    Формирует новый словарь с данными по вакансии.

    Args:
        vac_info (dict): словарь с ключами
    """
    dct_vac = dict.fromkeys(['vac_id', 'vac_region_name', 'job_name', 'salary', 'salary_min',
                        'salary_max', 'vac_company_name', 'vac_company_inn', 'duty', 
                        'phone', 'mobile_phone', 'email'], None)

    dct_vac['vac_id'] = vac_info.get('vacancy').get('id')
    dct_vac['vac_region_name'] = vac_info.get('vacancy').get('region').get('name', 'данных нет')
    dct_vac['vac_company_name'] = vac_info.get('vacancy').get('company').get('name', 'данных нет' )
    dct_vac['vac_company_inn'] = vac_info.get('vacancy').get('company').get('inn', 'данных нет')
    dct_vac['job_name'] = vac_info.get('vacancy').get('job-name')
    dct_vac['salary'] = vac_info.get('vacancy').get('salary', 'данных нет')
    dct_vac['salary_min'] = vac_info.get('vacancy').get('salary_min', 'данных нет')
    dct_vac['salary_max'] = vac_info.get('vacancy').get('salary_max', 'данных нет')
    dct_vac['duty'] = vac_info.get('vacancy').get('duty', 'данных нет')
    vacancy = vac_info.get('vacancy')
    dct_vac['phone'] = None
    dct_vac['mobile_phone'] = None
    dct_vac['email'] = None
    contact_list = vacancy.get('contact_list')
    for contact in contact_list:
        contact_type = contact.get('contact_type', 'данных нет').lower()
        if not dct_vac['phone'] and 'телефон' in contact_type and 'мобильный' not in contact_type:
            dct_vac['phone'] = contact.get('contact_value')
        elif not dct_vac['mobile_phone'] and 'мобильный телефон' in contact_type:
            dct_vac['mobile_phone'] = contact.get('contact_value')
        elif not dct_vac['email'] and ('почта' in contact_type or 'email' in contact_type):
            dct_vac['email'] = contact.get('contact_value')
    return dct_vac.copy()

4. функция -- для записи в файл

In [74]:
def write_book_to_file(name_file, dct_vac):
    '''
#     Записывает данные вакансии в CSV файл, 
#     автоматически добавляя заголовки при создании нового файла.
    
#     Args:
#         name_file (str): CSV файл для записи
#         dct_vac (dict): Словарь с данными вакансии (ключи станут заголовками столбцов)
    '''
    file_exists = os.path.isfile(name_file)
    with open(name_file, 'a', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=dct_vac.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(dct_vac)

5. Функция -- запускающая процесс

In [73]:

def pipeline_vac(name_file, offset_full:int, api_url:str, params_api:dict=None):
    '''
    Основная функция для сбора и сохранения вакансий из API
    
    Args:
        name_file (str): файл для сохранения результатов (CSV формат)
        offset_full (int): Общее количество страниц для сбора (пагинация)
        api_url (str): URL конечной точки API для получения вакансий
        params_api (dict, optional): Дополнительные параметры запроса. По умолчанию None
    '''
    for offset_value in range(0, offset_full):
        params_api.update({'offset': offset_value})

        # функция 1
        vacancies = send_get_request(api_url, params_api)
        # функция 2
        list_all_vac = get_list_vac(vacancies)
        if not list_all_vac:
            print(f'''\U0001F6AB Данные закончились на offset={offset_value}.
                  Обработано {offset_value} страниц ''')
            break    
        for vac in list_all_vac:
            vac_dat = get_vacansy(vac)
            print(f'''\U00002705 
                  Обрабатываю: {vac_dat.get('job_name', 'No_name')}''')
            #print(get_vacansy(vac))
            write_book_to_file(name_file, get_vacansy(vac))
          

Запуск(поменять ofsset_full = 85, если нужно собрать данные по всем страницам вакансий)

In [78]:
offset_full = 5
parametrs = {
    'region_code': 64}
api_url = "http://opendata.trudvsem.ru/api/v1/vacancies"

name_file = 'vc2.csv'

pipeline_vac(name_file, offset_full, api_url, parametrs)

Успешно, запрос отправлен на:  http://opendata.trudvsem.ru/api/v1/vacancies?region_code=64&offset=0
✅ 
                  Обрабатываю: Штукатур 5 разряда Отдела капитального строительства
✅ 
                  Обрабатываю: Инженер 1 категории Отдела подготовки и проведения закупок
✅ 
                  Обрабатываю: Электрогазосварщик 4 разряда транспортной службы
✅ 
                  Обрабатываю: Слесарь по ремонту автомобилей 3 разряда  Транспортной службы
✅ 
                  Обрабатываю: Курьер Службы эксплуатации пунктов редуцирования газа
✅ 
                  Обрабатываю: Подсобный рабочий 3 разряда Отдел капитального строительства
✅ 
                  Обрабатываю: Инженер-энергетик
✅ 
                  Обрабатываю: Мойщик посуды 2 разряда Столовой
✅ 
                  Обрабатываю: Газосварщик 5 разряда строительно-монтажного участка
✅ 
                  Обрабатываю: Слесарь аварийно-восстановительных работ 6 разряда Службы врезок и ремонта
✅ 
                  Обрабатываю: Слесарь п

Записываем csv-файл в DataFrame, просматриваем результат

In [83]:
dff = pd.read_csv('vc2.csv') 
dff.tail(40)


,vac_id,vac_region_name,job_name,salary,salary_min,salary_max,vac_company_name,vac_company_inn,duty,phone,mobile_phone,email
460,51432458-5899-11ef-8ab6-d549be31d974,Саратовская область,Оператор станков с программным управлением,от 44700,44700,44700,"АО ""НПП ""КОНТАКТ""",6453097665,фрезерная обработка деталей согласно чертежам.,+7(845) 235-78-56,NaN,kadry@kontakt-saratov.ru
461,56cadfb8-773a-11f0-b2d9-cb26dff57dd7,NaN,начальник смены,от 45000,45000,45000,"АО ""САРАТОВСКИЕ ОБОИ""",6454001398,"контроль за работой рабочих, за соблюдением пр...",+7(842) 229-54-12,NaN,IlyinaNV@saroboi.ru
462,6c5dd5a8-7760-11f0-a9f5-d549be31d974,Саратовская область,"Мастер контрольный (участка, цеха)",от 50000,50000,65000,"АО ""БРБФ""",6439102176,Обеспечение приемки деталей узлов и изделий в ...,+7(937) 636-93-91,NaN,personal_balrbf@mail.ru
463,82b473f8-089c-11f0-a533-cb26dff57dd7,Саратовская область,Фрезеровщик (фрезеровщик-универсал),от 23000,23000,43400,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,8455024525,NaN,kmzkadry@yandex.ru
464,f180d958-139a-11f0-a313-d549be31d974,Саратовская область,электромонтер по ремонту и обслуживанию электр...,от 36600,36600,36600,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,8455024525,NaN,kmzkadry@yandex.ru
465,fb9b5768-1392-11f0-a313-d549be31d974,Саратовская область,Токарь (токарь-универсал),от 23000,23000,40000,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,8455024525,NaN,kmzkadry@yandex.ru
466,bee9b440-1394-11f0-b544-e7d0d2cf29b1,Саратовская область,Наладчик холодноштамповочного оборудования,от 48000,48000,48000,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,8455024525,NaN,kmzkadry@yandex.ru
467,3c8a7e48-4061-11f0-9234-cb26dff57dd7,Саратовская область,Слесарь механосборочных работ (квота),от 23000,23000,37000,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,+7(845) 502-45-25,NaN,kmzkadry@yandex.ru
468,e7b7b098-1397-11f0-85c2-cb26dff57dd7,Саратовская область,Слесарь-ремонтник промышленного оборудования,от 23000,23000,42000,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,8455024525,NaN,kmzkadry@yandex.ru
469,cf4da0c8-405e-11f0-b9d8-d549be31d974,Саратовская область,Наладчик станков и манипуляторов с программным...,от 48000,48000,48000,"ОАО ""КМЗ""",6442005951,согласно должностной инструкции,+7(845) 502-45-25,NaN,kmzkadry@yandex.ru
